# 要素名

GRIB2 要素由三个数字 (GRIB Key) 确定：

- discipline
- parameterCategory
- parameterNumber

例如

| name | discipline | parameterCategory | parameterNumber |
|------|------------|-------------------|-----------------|
| temperature | 0          | 0                 | 0        |
| geopotential height | 0   | 3 | 5 |
| u component of wind | 0 | 2 | 2 |
| v component of wind | 0 | 2 | 3 |

更多要素编码可以参考 ECMWF 的 [Parameter Database](https://codes.ecmwf.int/grib/param-db/)。


`load_field_from_file()` 函数中的 `parameter` 参数支持字符串和字典两种格式。

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

from reki.data_finder import find_local_file
from reki.format.grib.eccodes import load_field_from_file

In [ ]:
start_time = pd.Timestamp.utcnow().floor(freq="D") - pd.Timedelta(days=2)
start_time_label = start_time.strftime("%Y%m%d%H")
forecast_time_label = "24h"
forecast_time = pd.to_timedelta(forecast_time_label)

gfs_grib2_orig_file_path = find_local_file(
    "cma_gfs_gmf/grib2/orig",
    start_time=start_time,
    forecast_time=forecast_time,
)
gfs_grib2_orig_file_path

## 字符串

`parameter` 参数可以指定作为要素场名的字符串，包括以下几种类型：

- ecCodes 支持的 `shortName` 要素名
- 内置的 WGRIB2 要素名
- 内置的 CEMC 要素名

### ecCodes shortName


GRIB2 是表格驱动的数据格式，数据文件中保存的元信息通常为数字形式，需要从数据文件之外的表格中查找数字对应的含义。
ecCodes 内置了一系列要素名，即 shortName，可以用来表示特定的要素场名称。

例如下面代码使用 `grib_ls` 命令列出 GRIB2 文件中的所有要素场信息，其中 `shortName` 可以作为 `parameter` 参数使用。

In [ ]:
from IPython.utils.capture import capture_output

with capture_output() as captured:
    !module load eccodes/2.29.0/intel && grib_ls -P count "{gfs_grib2_orig_file_path}" 2>/dev/null | head -n 22

print(f"{captured.stdout}")

```{note}
`unknown` 表示 ecCodes 无法识别该要素场，无法直接使用 shortName 来检索要素。
```

使用 ecCodes 的 shortName 可以用来检索要素场：

In [ ]:
field = load_field_from_file(
    gfs_grib2_orig_file_path,
    parameter="acpcp",
)
field.plot()

In [ ]:
field = load_field_from_file(
    gfs_grib2_orig_file_path,
    parameter="2t",
) - 273.15
field.plot()


```{note}
ecCodes 的 shortName 和 ecCodes 本身的版本有关系，不同版本的 shortName 可能不同，使用时请额外注意。
```

### WGRIB2

reki 也内置了 WGRIB2 使用的要素表格，支持 WGRIB2 要素名。

wgrib2 命令会打印 GRIB2 文件中的要素清单。

In [ ]:
with capture_output() as captured:
    !module load wgrib2/3.1.1/intel && wgrib2 "{gfs_grib2_orig_file_path}" 2>/dev/null | head -n 20

print(f"{captured.stdout}")

使用 WGRIB2 要素名称加载要素场：

In [ ]:
field = load_field_from_file(
    gfs_grib2_orig_file_path,
    parameter="TMP",
    level_type="heightAboveGround",
    level=2,
) - 273.15
field.plot()

In [ ]:
field = load_field_from_file(
    gfs_grib2_orig_file_path,
    parameter="APCP"
)
field.plot()

### CEMC要素清单

reki 内置了 CEMC 的要素清单，支持 CEMC 自定义的变量名。

In [ ]:
from reki.format.grib.config import CEMC_PARAM_TABLE

CEMC_PARAM_TABLE.head(n=10)

使用 CEMC 要素名可以方便地加载一些 CEMC 定义的变量。
下面代码加载辐射亮温（红外通道）要素场：

In [ ]:
field = load_field_from_file(
    gfs_grib2_orig_file_path,
    parameter="bti"
)
field.plot()

加载 2 米温度场

In [ ]:
field = load_field_from_file(
    gfs_grib2_orig_file_path,
    parameter="t2m"
) - 273.15
field.plot()

加载 0-3km 垂直风切变

In [ ]:
field = load_field_from_file(
    gfs_grib2_orig_file_path,
    parameter="shr(0-3000)"
)
field.plot()

## 字典格式

parameter 参数可以直接设置字典格式的 GRIB Key。

比如雷达反射率的编码如下：

```
discipline = 0
parameterCategory = 16
parameterNumber = 225
```

In [ ]:
field = load_field_from_file(
    gfs_grib2_orig_file_path,
    parameter={
        "discipline": 0,
        "parameterCategory": 16,
        "parameterNumber": 225,
    },
    level_type="pl",
    level=850
)
field.plot()

```{note}
理论上 `parameter` 参数中可以设置任意 GRIB Key，不过建议将不同 GRIB Key 分散到不同的参数中。
`load_field_from_file` 函数也支持任意 GRIB key 作为关键字参数。
```